In [ ]:
# Connect my Google drive
from google.colab import drive

drive.mount('/gdrive')
gdrive_root = '/gdrive/My Drive/Research/Colab Notebooks'

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import numpy as np
import os
import random
import torchvision
import torchvision.transforms as transforms
import PIL
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Choose Hyper-parameters(training & optimization) 여기도 전부다 고정해도댐
max_epoch = 20
batch_size = 256 # normal batch_size
learning_rate = 0.001
output_dim = 10
p = 0.5
device = 'cuda'

In [ ]:
#불러온 CIFAR10 이미지를 모델이 학습하기 쉽도록 만져주는 코드
#randomcorp은 이미지 크기를 잘라주는 코드고 RandomHorizontalFlip은 좌우 반전을 해주는 코드(기본값은 50프로고 확률 조정 가능 말그대로 random으로 좌우반전 해주는 것 )
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
#테스트데이터의 경우는 위처럼 크기를 만져주지 않음, 그대로 테스트할 것 이기 때문
transform_test = transforms.Compose([
    transforms.ToTensor(),
])

data_dir = os.path.join(gdrive_root, 'cifar10') # 이거도 너 데이터 어디다 넣을껀지, 데이터란건 cifar10을 저장할건데 어디다 할건지임 그걸 위에 칸에서 수정하면댐
if not os.path.exists(data_dir):
  os.makedirs(data_dir)
  
#아래는 CIFAR10 dataset을 불러오는 코드
train_dataset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # resnet18의 경우 3x3 필터를 사용 첫번째 conv에서만 너비와 높이를 반으로 줄임
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity값인 x를 두번째 conv가 끝나고 나면 더해주고 그다음 relu함수로 보냄
        if stride != 1: # stride가 1이 아니라면, 즉 identity의 배열이 두번째 conv가 끝난 f(x)의 배열과 다를경우 맞춰줄수 있도록 하는 코드
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection, 층을 너무 깊이 쌓거나 노드 수를 너무 크게 증가시키면 입력 정보가 여러 층을 거치면서 이전 층에 대한 정보 손실이 발생할 수 있고 가중치가 잘못된 방향으로 갱신되는 문제가 발생할 수 있음 
                                # 그래서 이전 층의 정보를 이용하기 위해 이전 층의 정보를 연결하는 잔차 연결(skip connection)을 적용함
        out = F.relu(out)
        return out


# block들을 resnet의 blcok이 들어갈 자리에 대입해주어서 resnet을 구성한다
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# 위에서 만든 resnet 클래스에 대입한 block들을 두번씩 써주어야 하므로 2번씩 써주는 코드
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
my_classifier = ResNet18() #resnet18을 불러오고
my_classifier = my_classifier.to(device) #불러온 renet18을 cuda로 돌릴거임
print(my_classifier)
my_classifier = torch.nn.DataParallel(my_classifier) #모델을 병렬로 실행하여 gpu에서 돌릴거임
cudnn.benchmark = True #쿠다가 다중회선 알고리즘을 벤치마킹하여 가장 좋은 알고리즘을 선택하게 해준다
optimizer = optim.Adam(my_classifier.parameters(), lr=learning_rate) 

In [ ]:
# 너가 학습시킨 파라미터를 저장하는 곳인데 경로 잘 지정하면 거기에 학습된 파라미터가 저장되고 나중에 다시 불러올 수 있음(load_state라는 코드가 보일거임 이걸로 불러오는거)
ckpt_dir = os.path.join(gdrive_root, 'checkpoints')
if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)

best_acc = 0.
ckpt_path = os.path.join(ckpt_dir, 'lastest.pt')
if os.path.exists(ckpt_path):
  ckpt = torch.load(ckpt_path)
  try:
    my_classifier.load_state_dict(ckpt['my_classifier'])
    optimizer.load_state_dict(ckpt['optimizer'])
    best_acc = ckpt['best_acc']
  except RuntimeError as e:
    print('wrong checkpoint')
  else:
    print('checkpoint is loaded !')
    print('current best accuracy : %.2f' % best_acc) # 소수점 2번째까지 나타낸다.

In [ ]:
# train과정이고 하나씩 읽어보면 무리는 없을텐데 혹시 모르니까 몇개 주석 달아둠

it = 0
train_losses = [] # loss의 보관장소 나중에 시각화를 위함
test_losses = []
for epoch in range(max_epoch):
  # train phase
  my_classifier.train()
  for inputs, labels in train_loader: # dataloader인데 3번째 칸에서 set해놓은거고 나중에는 이거도 짜야댐 일단은 그냥 가져온다고 생각 근데 어떤 형태로 가져오는지는 체크해봐
    it += 1
    inputs = inputs.to(device) # dataloader로 받은 data gpu에 올리는데 이건 network돌릴때 data랑 network랑 둘다 gpu에 있어야 돌아가기 때문에 얘도 올려줘야댐
    labels = labels.to(device) # 위랑 같은이유
    logits = my_classifier(inputs) # network output
    loss = F.cross_entropy(logits, labels) # loss 계산시 cross_entropy로 계산
    optimizer.zero_grad() # optimizer 초기화인데 이전 step에서 정해진 optimizer값이 다음스텝에서 반영되면 안되니까 초기화
    loss.backward() # 이거하면 back propagation이 모든 과정에서 자동화됨
    optimizer.step() # 이거하면 그 back propagation을 adam이란 우리가 설정한 optimizer로 자동 update해줌
    acc = (logits.argmax(dim=1) == labels).float().mean() # 각 batch별로 평균을 출력 여기에서 출력되는 것은 각 epoch의 마지막에 걸려있는 batch

  # save losses in a list so that we can visualize them later.
  train_losses.append(loss)  

  # test phase / 여기선 당연한 소리지만 측정만 해야하니까 update 과정이 있으면 안댐
  n = 0.
  test_loss = 0.
  test_acc = 0.
  my_classifier.eval()
  for test_inputs, test_labels in test_loader:
    test_inputs = test_inputs.to(device)
    test_labels = test_labels.to(device)

    logits = my_classifier(test_inputs)
    test_loss += F.cross_entropy(logits, test_labels, reduction='sum').item() 
    test_acc += (logits.argmax(dim=1) == test_labels).float().sum().item() # 각 epoch별로 평균을 출력 여기에서 출력되는 것은 각 epoch의 모든 data의 평균 acc(전체 test dataset에서의 acc)
    n += test_inputs.size(0)

  test_loss /= n
  test_acc /= n
  test_losses.append(test_loss)

  print('[epoch:{}, iteration:{}] train loss : {:.4f} train accuracy : {:.4f}'.format(epoch, it, loss.item(), acc.item())) # loss가 어떻게 구해지는질 잘 모르겠다.(acc와는 다르게 각 data의 pixel별로 loss가 있기에)
  print('[epoch:{}, iteration:{}] test_loss : {:.4f} test accuracy : {:.4f}'.format(epoch, it, test_loss, test_acc)) 


  # save checkpoint whenever there is improvement in performance / 여기가 아까 위에서 적었던 check point를 더 좋았던 학습의 parameter로 바꾸는거임
  if test_acc > best_acc:
    best_acc = test_acc
    ckpt = {'net':my_classifier.state_dict(),
              'optimizer':optimizer.state_dict(),
              'best_acc':best_acc}
    # torch.save(ckpt, ckpt_path)
    print('checkpoint is saved !')

In [ ]:
# 여기는 그래프 그리는 과정인데 시각화는 좀 더 잘 배우면 좋음 특히 민수는 개잘해야되는걸로 알아

# plt.plot(train_losses, label='train loss') # 그래프 x, y축이 어떤 data에 대한 수치인지 표시하기 / 위에서 train 안하려면 plt쓰는 부분 주석처리
plt.plot(test_losses, label='test loss')
plt.legend()


# Testing(best model로 단 한번한 test하는 것)
n = 0.
test_loss = 0.
test_acc = 0.
my_classifier.eval()
for test_inputs, test_labels in test_loader:
  test_inputs = test_inputs.to(device)
  test_labels = test_labels.to(device)

  logits = my_classifier(test_inputs)
  test_loss += F.cross_entropy(logits, test_labels, reduction='sum').item()
  test_acc += (logits.argmax(dim=1) == test_labels).float().sum().item()
  n += test_inputs.size(0) # = batch size

test_loss /= n
test_acc /= n

print('Test_loss : {:.4f}, Test accuracy : {:.4f}'.format(test_loss, test_acc))

In [ ]:
# 사실 여기도 그냥 이미지 하나 뽑아서 잘 예측했나 보는건데 필요 없는 칸임 (말그대로 체크용 코드)

my_classifier.eval()

num_test_samples = len(test_dataset)
random_idx = random.randint(0, num_test_samples)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

test_input, test_label = test_dataset.__getitem__(random_idx) # dataset에서 꺼내올때 getitem이 필요한듯
test_prediction = F.softmax(my_classifier(test_input.unsqueeze(0).to(device)), dim=1).argmax().item() # 행에 대하여 softmax 함수적용, 4차원 data용 network여서 배치개수 1인 느낌으로 unsqueeze추가
print('label : %s' % classes[test_label])
print('prediction : %s' % classes[test_prediction])

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# show images
imshow(torchvision.utils.make_grid(test_input)) # 이미지 가공후 imshow로 넣어주기